In [1]:
from core import init

# Uncomment for requests caching
# init()

In [1]:
import requests

In [2]:
from search import search

steam_results = list(search("Singularity 6 Palia Steam", num=1))
steam_results

2024-08-20 10:18:43.504 | DEBUG    | search:search:65 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - Singularity 6 Palia Steam', 'totalResults': '20000', 'searchTerms': 'Singularity 6 Palia Steam', 'count': 1, 'startIndex':

[SearchResult(title='Palia on Steam', link='https://store.steampowered.com/app/2707930/Palia/', snippet='Palia. Developer. Singularity 6 Corporation. Publisher. Singularity 6 Corporation. Released. Mar 25, 2024. Palia is a vibrant new world made just for you. Craft\xa0...', formattedUrl='https://store.steampowered.com/app/2707930/Palia/')]

In [25]:
import re
from typing import Optional

STEAM_URL_PATTERN = re.compile(r"https://store.steampowered.com/app/(\d+)/(\w+)/")

def extract_steam_id(url: str) -> Optional[int]:
    if STEAM_URL_PATTERN.match(url):
        return int(STEAM_URL_PATTERN.match(url).group(1))
    return None

extract_steam_id("https://store.steampowered.com/app/2707930/Palia/")

2707930

In [23]:
from typing import Iterable, List, Optional
from pydantic import BaseModel

class Author(BaseModel):
    last_played: int
    num_games_owned: int
    num_reviews: int
    playtime_at_review: int
    playtime_forever: int
    playtime_last_two_weeks: int
    steamid: str

class Review(BaseModel):
    author: Author
    comment_count: int
    hidden_in_steam_china: bool
    language: str
    received_for_free: bool
    recommendationid: str
    review: str
    steam_china_location: str
    steam_purchase: bool
    timestamp_created: int
    timestamp_updated: int
    voted_up: bool
    votes_funny: int
    votes_up: int
    weighted_vote_score: str
    written_during_early_access: bool

    developer_response: Optional[str] = None
    timestamp_dev_responded: Optional[int] = None


class QuerySummary(BaseModel):
    num_reviews: int

    # These are only on the first page
    review_score: Optional[int] = None
    review_score_desc: Optional[str] = None
    total_positive: Optional[int] = None
    total_negative: Optional[int] = None
    total_reviews: Optional[int] = None

class SteamResponse(BaseModel):
    cursor: str
    query_summary: QuerySummary
    reviews: List[Review]
    success: int

def get_reviews(steam_id: int, num_reviews=100) -> Iterable[Review]:
    num_per_page = 100 if num_reviews > 100 else num_reviews

    reviews_collected = 0
    cursor = "*"

    while reviews_collected < num_reviews:
        response = requests.get(
            f"https://store.steampowered.com/appreviews/{steam_id}", 
            params={
                "json": 1, 
                "language": "english", 
                "purchase_type": "all", 
                "num_per_page": num_per_page,
                "cursor": cursor, 
                # "filter": "recent", 
                # "review_type": "all", 
                # "cursor": "*", 
                # "day_range": 365, 
                # "filter_offtopic_activity": 0
                })
        response.raise_for_status()

        response_data = SteamResponse(**response.json())

        # Nothing to emit
        if not response_data.success or not response_data.reviews:
            break

        yield from response_data.reviews

        # We got partial results, which is a sign it's the last page
        if response_data.query_summary.num_reviews < num_per_page:
            break

        cursor = response_data.cursor

    return response_data.reviews

palia_id = 2707930
dota_id = 10

reviews = list(get_reviews(palia_id))

from pprint import pprint
pprint(list(reviews))

[Review(author=Author(last_played=1724180425, num_games_owned=2, num_reviews=1, playtime_at_review=1818, playtime_forever=2094, playtime_last_two_weeks=2094, steamid='76561199353237050'), comment_count=0, hidden_in_steam_china=True, language='english', received_for_free=False, recommendationid='172626263', review='very good game, fun game\r\n', steam_china_location='', steam_purchase=False, timestamp_created=1724161259, timestamp_updated=1724161259, voted_up=True, votes_funny=0, votes_up=1, weighted_vote_score='0.509803950786590576', written_during_early_access=False, developer_response=None, timestamp_dev_responded=None),
 Review(author=Author(last_played=1724122904, num_games_owned=0, num_reviews=2, playtime_at_review=2994, playtime_forever=2994, playtime_last_two_weeks=2994, steamid='76561198811551444'), comment_count=0, hidden_in_steam_china=True, language='english', received_for_free=False, recommendationid='172497287', review='LOVE LOVE LOVE\n\nIf You Enjoy- Minecraft, Sims, Star

In [27]:
# Just markdown format a bunch of them
from datetime import datetime

for review in reviews:
    review_dt = datetime.fromtimestamp(review.timestamp_created)
    print(f"""
### {review.author.steamid} {'Thumbs Up' if review.voted_up else 'Thumbs Down'} ({review_dt.strftime('%Y-%m-%d')})
{review.review.strip()}
""")


### 76561199353237050 Thumbs Up (2024-08-20)
very good game, fun game


### 76561198811551444 Thumbs Up (2024-08-19)
LOVE LOVE LOVE

If You Enjoy- Minecraft, Sims, Stardew Valley, Questing in General

Than this Game is for You. This is probably my first Steam Review and this Game has Earned it!!!


### 76561199037913524 Thumbs Up (2024-08-19)
i really like palia :3 
i think its a good game for winding down, and i really like how its not based on real time events or whatever. you can do everything at your own pace!! 
the character designs are really great, the world itself is super pretty, and i really love most of the mechanics for farming/cooking/etc
super chill and super cozy ^_^


### 76561199527873206 Thumbs Down (2024-08-19)
Initially played with my husband, though he checked out in the first 2 hours due to slow story progression, which wasn't really an issue for me. Little did I know, 79 hours later I'm stuck with 3 uncompleted quests, 2 of which now say 'wait for the story to b